In [2]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename='', scripts="run_polygraph_2.py"):
    full_config = 'cd ../'
    j = 0
    n_devices = len(cuda_devices)
    for i, mc_configs in enumerate(configs):
        for conf in expand_config(mc_configs):
            for task_name in tasks:
                conf["task"] = task_name
                if not ("12b" in conf["model"]):
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python {scripts}'
                else:
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]},{cuda_devices[(j+1)%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python {scripts}'
                    j+=1
                    
                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                new_task += f' {args}'
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [4]:
tasks = ['coqa', 'sciq', 'triviaqa', 'truthfullqa', 'mmlu']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [1000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],# 'stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_9'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1.sh')

TypeError: generate_train_str_config() missing 1 required positional argument: 'task_name'

In [14]:
tasks = ['gsm8k', 'xsum', 'wmt14_deen', 'medquad', 'cnn', 'samsum', 'pubmedqa']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [1000],
    'subsample_background_train_dataset': [500],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],# 'stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_9'],
    'samples_n': [5],
}
    
cuda_devices = [0,1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p2.sh')

KeyError: 'subsample_gsm8k_train_dataset'

In [4]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu', 'samsum', 'pubmedqa', 'wmt14_deen', 'cnn']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1_1.sh')

In [5]:
tasks = ['gsm8k', 'xsum', 'medquad']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['google/gemma-7b', 'meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p2_1.sh')

In [6]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu', 'samsum', 'pubmedqa', 'wmt14_deen', 'gsm8k', 'xsum', 'medquad', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p3_1.sh')

In [7]:
tasks = ['pubmedqa', 'wmt14_deen', 'cnn']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1_2.sh')

In [10]:
tasks = ['gsm8k', 'xsum']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    "subsample_gsm8k_train_dataset": [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p3_2.sh')

In [41]:
tasks = ['medquad', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    "subsample_gsm8k_train_dataset": [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['stabilityai/stablelm-2-12b'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_3'],
    'samples_n': [5],
}
    
cuda_devices = [1,2,3,4]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p3_3.sh')

In [37]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu', 'samsum', 'pubmedqa', 'wmt14_deen', 'gsm8k', 'xsum', 'medquad', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [-1],
    'subsample_background_train_dataset': [10],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_saplma'],
    "subsample_gsm8k_train_dataset": [2000],
    'samples_n': [5],
    "use_truefalse_dataset": [True]
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p4_1.sh', scripts="run_polygraph_3.py")

In [38]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu', 'samsum', 'pubmedqa', 'wmt14_deen', 'gsm8k', 'xsum', 'medquad', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_saplma_ds'],
    "subsample_gsm8k_train_dataset": [2000],
    'samples_n': [5],
    "use_truefalse_dataset": [False],
}
    
cuda_devices = [3]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p4_2.sh', scripts="run_polygraph_3.py")

In [8]:
tasks = ['sciq', 'coqa', 'triviaqa', 'truthfullqa', 'mmlu']

layers = {"google/gemma-7b": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),
          "meta-llama/Meta-Llama-3-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)]))}

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal'],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p5_1.sh', scripts="run_polygraph_4.py")

In [14]:
tasks = ['gsm8k', 'xsum', 'medquad']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal'],
    'samples_n': [5],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p5_2.sh', scripts="run_polygraph_4.py")

In [11]:
tasks = ['samsum', 'pubmedqa', 'wmt14_deen', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3-8B'],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal'],
    'samples_n': [5],
}
    
cuda_devices = [2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p5_3.sh', scripts="run_polygraph_4.py")

## Final experiments

In [11]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ignore_exceptions=False use_density_based_ue=True')
    
    config_list.append('batch_size={}'.format(config['batch_size']))

    if ("gsm8k" in config["task"]) or ("medquad" in config["task"]) or ("samsum" in config["task"]):
        config_list.append('subsample_train_dataset={}'.format(config['subsample_gsm8k_train_dataset']))
    else:
        config_list.append('subsample_train_dataset={}'.format(config['subsample_train_dataset']))
    if config.get("ablation", False):
        if ("gsm8k" in config["task"]) or ("medquad" in config["task"]) or ("samsum" in config["task"]):
            config_list.append('subsample_background_train_dataset={}'.format(config['subsample_gsm8k_train_dataset']))
        else:
            config_list.append('subsample_background_train_dataset={}'.format(config['subsample_train_dataset']))
    else:
        config_list.append('subsample_background_train_dataset={}'.format(config['subsample_background_train_dataset']))
    config_list.append('subsample_eval_dataset={}'.format(config['subsample_eval_dataset']))
    config_list.append('model.path={}'.format(config['model']))
    
    if ("gemma" in config['model']) or ("mistral" in config['model'].lower()) or ("llama-3" in config['model'].lower()) or ("stablelm-2" in config['model'].lower()):
        config_list.append('+model.attn_implementation=eager')
    if ("cache_path" in config.keys()) and config.get("ablation", False):
        config_list.append('cache_path={}_{}'.format(config['cache_path'], config['subsample_train_dataset']))
    elif ("cache_path" in config.keys()):
        config_list.append('cache_path={}'.format(config['cache_path']))
    if ("metric_thrs" in config.keys()):
        config_list.append('+metric_thrs={}'.format(config['metric_thrs']))
    if ("layers" in config.keys()):
        config_list.append('+layers={}'.format(config['layers'][config['model']]))
    if ("samples_n" in config.keys()):
        config_list.append('+generation_params.samples_n={}'.format(config['samples_n']))
    if ("baselines" in config.keys()):
        if config['baselines'] == "all":
            config_list.append('+run_baselines=True')
            config_list.append('+md_device=cuda')
            config_list.append('+run_proposed_methods=True')
            config_list.append('+clean_md_device=cuda')
        elif config['baselines']:
            config_list.append('+run_baselines=True')
            config_list.append('+md_device=cuda')
        else:
            config_list.append('+run_proposed_methods=True')
            config_list.append('+run_layerwise_methods=True')
            config_list.append('+clean_md_device=cuda')
            config_list.append('+md_device=cuda')
            config_list.append('+run_eigenscore=False')
                
    if ("use_truefalse_dataset" in config.keys()):
        config_list.append('+use_truefalse_dataset={}'.format(config['use_truefalse_dataset']))
        if (config["use_truefalse_dataset"]):
            config_list.append('train_dataset=../data/publicDataset/truefalsedata.csv +train_text_column=statement +train_label_column=label')
    return config_list

In [9]:
layers = {"google/gemma-2-9b": "\"[{},-1]\"".format(",".join([str(x) for x in range(41)])),
          "meta-llama/Meta-Llama-3.1-8B": "\"[{},-1]\"".format(",".join([str(x) for x in range(31)])),
          "stabilityai/stablelm-2-12b": "\"[{},-1]\"".format(",".join([str(x) for x in range(39)])),
          "Qwen/Qwen2.5-7B": "\"[{},-1]\"".format(",".join([str(x) for x in range(27)])),}

In [8]:
tasks = ['sciq', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'samsum']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "Qwen/Qwen2.5-7B"],# "stabilityai/stablelm-2-12b"],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'baselines': ["all"],#[False, True],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_final_1.sh', scripts="run_polygraph.py")

In [9]:
tasks = ['gsm8k', 'medquad', 'xsum']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "Qwen/Qwen2.5-7B"],# "stabilityai/stablelm-2-12b"],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'baselines': ["all"],#[False, True],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'samples_n': [5],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_final_2.sh', scripts="run_polygraph.py")

In [10]:
tasks = ['pubmedqa', 'cnn', "wmt19_deen"]# 'wmt14_fren', 

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "Qwen/Qwen2.5-7B"],# "stabilityai/stablelm-2-12b"],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'baselines': ["all"],#False, True],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'samples_n': [5],
}
    
cuda_devices = [2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_final_3.sh', scripts="run_polygraph.py")

In [7]:
tasks = ['pubmedqa', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'baselines': ["all"],#False, True],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'samples_n': [5],
}
    
cuda_devices = [2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_final_3_2.sh', scripts="run_polygraph.py")

In [12]:
tasks = ['medquad', 'gsm8k']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'baselines': ["all"],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'samples_n': [5],
}
    
cuda_devices = [1,2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_final_3_3.sh', scripts="run_polygraph.py")

### blackbox

In [12]:
tasks = ['sciq', 'truthfullqa', 'samsum', 'gsm8k', 'cnn', 'triviaqa', 'mmlu']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_gsm8k_train_dataset': [1000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B"],# "stabilityai/stablelm-2-12b"],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'baselines': [False],#[False, True],
    'samples_n': [5],
}
    
cuda_devices = ['0,1']

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_proxy_1.sh', scripts="run_polygraph_test.py")

## train size ablation

In [60]:
tasks = ['sciq']

train_configs = {
    'batch_size': [1],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'subsample_train_dataset': [100, 200, 500, 1000, 2000],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_ablation'],
    'baselines': [False],
    'ablation': [True],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_abl_1.sh', scripts="run_polygraph.py")

In [61]:
tasks = ['truthfullqa']

train_configs = {
    'batch_size': [1],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'subsample_train_dataset': [100, 200, 500],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_ablation'],
    'baselines': [False],
    'ablation': [True],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_abl_1_1.sh', scripts="run_polygraph.py")

In [62]:
tasks = ['mmlu', 'cnn']

train_configs = {
    'batch_size': [1],
    'subsample_gsm8k_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'subsample_train_dataset': [100, 200, 500, 1000, 2000],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'baselines': [False],
    'ablation': [True],
    'samples_n': [5],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_abl_2.sh', scripts="run_polygraph.py")

In [66]:
tasks = ['medquad']

train_configs = {
    'batch_size': [1],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'subsample_gsm8k_train_dataset': [100, 200, 500, 1000, 2000],
    'subsample_train_dataset': [100],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'baselines': [False],
    'ablation': [True],
    'samples_n': [5],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_abl_3.sh', scripts="run_polygraph.py")

In [67]:
tasks = ['gsm8k']

train_configs = {
    'batch_size': [1],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B", "google/gemma-2-9b"],# "stabilityai/stablelm-2-12b"],
    'subsample_gsm8k_train_dataset': [100, 200, 500, 1000, 2000],
    'subsample_train_dataset': [100],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'baselines': [False],
    'ablation': [True],
    'samples_n': [5],
}
    
cuda_devices = [2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_abl_4.sh', scripts="run_polygraph.py")

In [74]:
tasks = ['samsum']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ["meta-llama/Meta-Llama-3.1-8B"],# "stabilityai/stablelm-2-12b"],
    'subsample_gsm8k_train_dataset': [100, 200, 500, 1000],
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    'cache_path': ['./workdir/output_layers_internal_final'],
    'baselines': [False],
    'ablation': [True],
    'samples_n': [5],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_abl_5.sh', scripts="run_polygraph.py")

## Generalization

In [9]:
import yaml

def generate_train_str_config(config, task_name):
    config_list = []
    config_list.append('ignore_exceptions=False use_density_based_ue=True')
    
    config_list.append('batch_size={}'.format(config['batch_size']))
    config_list.append('train_test_split=False')
    config_list.append('subsample_train_dataset={}'.format(config['subsample_train_dataset']))
    config_list.append('subsample_background_train_dataset={}'.format(config['subsample_background_train_dataset']))
    config_list.append('subsample_eval_dataset={}'.format(config['subsample_eval_dataset']))
    config_list.append('model.path={}'.format(config['model']))
    if ("gemma" in config['model']) or ("mistral" in config['model'].lower()) or ("llama-3" in config['model'].lower()) or ("stablelm-2" in config['model'].lower()):
        config_list.append('+model.attn_implementation=eager')
    if ("cache_path" in config.keys()) and not config.get("upd_path", False):
        config_list.append('cache_path={}'.format(config['cache_path']))
    if ("samples_n" in config.keys()):
        config_list.append('+generation_params.samples_n={}'.format(config['samples_n']))
    if ("metric_thrs" in config.keys()):
        config_list.append('+metric_thrs={}'.format(config['metric_thrs']))
    if ("layers" in config.keys()):
        config_list.append('+layers={}'.format(config['layers'][config['model']]))
    if config.get("generalization", False):
        try:
            ood_cfg = gen_tasks[task_name][config["exp_idx"]]
        except:
            return ""
        for i, ds in enumerate(ood_cfg):
            if ("cache_path" in config.keys()) and config.get("upd_path", False):
                config_list.append('cache_path={}_on_{}'.format(config['cache_path'], ds))
            with open(f"../configs/polygraph_eval_{ds}.yaml") as stream:
                gen_config = yaml.safe_load(stream)
            config_list.append('+max_new_tokens_{}={}'.format(i+1, gen_config['max_new_tokens']))
            config_list.append('+train_dataset_{}=\"{}\"'.format(i+1, gen_config['dataset']))
            config_list.append('+train_text_column_{}={}'.format(i+1, gen_config['text_column']))
            config_list.append('+train_label_column_{}={}'.format(i+1, gen_config['label_column']))
            config_list.append('+train_prompt_{}=\"{}\"'.format(i+1, gen_config['prompt']))
            
            config_list[-1] = config_list[-1].replace("\n", "\\n")
            config_list[-1] = config_list[-1].replace("$", "\\$")
            config_list[-1] = config_list[-1].replace("'s", " is")
            config_list[-1] = config_list[-1].replace("(", "\\(")
            config_list[-1] = config_list[-1].replace(")", "\\)")
            config_list[-1] = config_list[-1].replace("}", "\\}")
            config_list[-1] = config_list[-1].replace("{", "\\{")
            config_list[-1] = config_list[-1].replace("]", "\\]")
            config_list[-1] = config_list[-1].replace("[", "\\[")
            config_list[-1] = config_list[-1].replace(",", "\\,")
            config_list[-1] = config_list[-1].replace("=", "\\=")
            # config_list[-1] = config_list[-1].replace("+", "\\+")
            config_list.append('+train_split_{}={}'.format(i+1, gen_config['train_split']))

            if "description" in gen_config.keys():
                config_list.append("+train_description_{}=\"{}\"".format(i+1, gen_config['description']))
                config_list[-1] = config_list[-1].replace("\n", "\\n")
                config_list[-1] = config_list[-1].replace("'s", " is")
                config_list[-1] = config_list[-1].replace("(", "\\(")
                config_list[-1] = config_list[-1].replace(")", "\\)")
                config_list[-1] = config_list[-1].replace("}", "\\}")
                config_list[-1] = config_list[-1].replace("{", "\\{")
                config_list[-1] = config_list[-1].replace(",", "\\,")
                config_list[-1] = config_list[-1].replace("$", "\\$")
                config_list[-1] = config_list[-1].replace("=", "\\=")
                config_list[-1] = config_list[-1].replace("]", "\\]")
                config_list[-1] = config_list[-1].replace("[", "\\[")
                # config_list[-1] = config_list[-1].replace("+", "\\+")
                
            if "few_shot_split" in gen_config.keys():
                config_list.append('+few_shot_split_{}={}'.format(i+1, gen_config['few_shot_split']))
            if "n_shot" in gen_config.keys():
                config_list.append('+train_n_shot_{}={}'.format(i+1, gen_config['n_shot']))
    if ("baselines" in config.keys()):
        if config['baselines'] == "all":
            config_list.append('+run_baselines=False')
            config_list.append('+run_eigenscore=False')
            config_list.append('+md_device=cuda')
            config_list.append('+run_proposed_methods=True')
            config_list.append('+run_layerwise_methods=True')
            config_list.append('+clean_md_device=cuda')
        elif config['baselines']:
            config_list.append('+run_baselines=False')
            config_list.append('+run_eigenscore=False')
            config_list.append('+run_layerwise_methods=True')
            config_list.append('+md_device=cuda')
        else:
            config_list.append('+run_proposed_methods=True')
            config_list.append('+clean_md_device=cuda')
    if "is_ood" in config.keys():
        config_list.append('+is_ood={}'.format(config['is_ood']))
        
    return config_list

In [10]:
def generate_bash(configs, cuda_devices, tasks, generate_func, script_name="polygraph_eval", filename='', n_gpus=1):
    full_config = 'cd ../'
    j = 0
    print(tasks)
    n_devices = len(cuda_devices)
    for i, mc_configs in enumerate(configs):
        for conf in expand_config(mc_configs):
            for task_name in tasks:
                
                if (n_gpus == 1) or ((task_name not in ["gsm8k", "xsum", "medquad"]) and (("7b" in conf["model"]) or ("8b" not in conf["model"]))):
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python run_polygraph.py'
                else:
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]},{cuda_devices[(j+1)%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python run_polygraph.py'
                    j+=1
                    
                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf, task_name))
                if not len(args.strip()):
                    continue
                new_task += f' {args}'
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [11]:
datasets_ts = ['samsum', 'xsum', 'cnn']
datasets_tr = ['wmt14_fren', "wmt19_deen"]
datasets_qa_s = ['sciq', 'coqa', 'triviaqa', 'mmlu']
# datasets_qa_l = ['truthfullqa', 'pubmedqa', 'medquad']
datasets_qa_l = ['truthfullqa', 'pubmedqa']
all_tasks = [datasets_ts, datasets_qa_s, datasets_qa_l]#, datasets_tr]

In [49]:
import numpy as np
import copy

n_exps = 1
tasks = ['truthfullqa', 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [1000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B'],
    'cache_path': ['./workdir/gen_output_loo_new'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_v2_all.sh')

{'truthfullqa': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'pubmedqa']], 'samsum': [['xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa']]}
['truthfullqa', 'samsum']


In [33]:
import numpy as np
import copy

n_exps = 1
tasks = ['mmlu', 'truthfullqa', 'samsum', 'sciq']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [400],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B'],
    'cache_path': ['./workdir/gen_output_loo_new'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_v2_1.sh')

{'mmlu': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'truthfullqa', 'pubmedqa', 'medquad']], 'truthfullqa': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'pubmedqa', 'medquad']], 'samsum': [['xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa', 'medquad']], 'sciq': [['samsum', 'xsum', 'cnn', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa', 'medquad']]}
['mmlu', 'truthfullqa', 'samsum', 'sciq']


In [34]:
import numpy as np
import copy

n_exps = 1
tasks = ['mmlu', 'truthfullqa', 'samsum', 'sciq']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    for i, ds_tasks in enumerate(all_tasks):
        if task in ds_tasks:
            for ds in ds_tasks:
                if ds != task:
                    ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [400],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B'],
    'cache_path': ['./workdir/gen_output_loo_new'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_v2_2.sh')

{'mmlu': [['sciq', 'coqa', 'triviaqa']], 'truthfullqa': [['pubmedqa', 'medquad']], 'samsum': [['xsum', 'cnn']], 'sciq': [['coqa', 'triviaqa', 'mmlu']]}
['mmlu', 'truthfullqa', 'samsum', 'sciq']


In [9]:
import numpy as np
import copy

n_exps = 1
tasks = ['mmlu', 'truthfullqa', 'samsum', 'sciq']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    for i, ds_tasks in enumerate(all_tasks):
        if task in ds_tasks:
            for ds in ds_tasks:
                if ds != task:
                    ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [1000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B'],
    'cache_path': ['./workdir/gen_output_loo_new'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_v2_3.sh')

{'mmlu': [['sciq', 'coqa', 'triviaqa']], 'truthfullqa': [['pubmedqa', 'medquad']], 'samsum': [['xsum', 'cnn']], 'sciq': [['coqa', 'triviaqa', 'mmlu']]}
['mmlu', 'truthfullqa', 'samsum', 'sciq']


In [37]:
import numpy as np
import copy

n_exps = 3
tasks = ['mmlu', 'truthfullqa', 'samsum', 'sciq']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    for i, ds_tasks in enumerate(all_tasks):
        if task in ds_tasks:
            for ds in ds_tasks:
                if ds != task:
                    # ds_exps.append(ds)
                    gen_tasks[task].append([ds])
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [2000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B'],
    'cache_path': ['./workdir/gen_output_loo_new'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
    "upd_path": [True]
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_v2_4.sh')

{'mmlu': [['sciq'], ['coqa'], ['triviaqa']], 'truthfullqa': [['pubmedqa'], ['medquad']], 'samsum': [['xsum'], ['cnn']], 'sciq': [['coqa'], ['triviaqa'], ['mmlu']]}
['mmlu', 'truthfullqa', 'samsum', 'sciq']


### old

In [10]:
import numpy as np
import copy

n_exps = 1
tasks = ['mmlu']#, 'truthfullqa', 'samsum', 'cnn', 'medquad']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    # for i, ds_tasks in enumerate(all_tasks):
    #     if task in ds_tasks:
    #         for ds in ds_tasks:
    #             if ds != task:
    #                 ds_exps.append(ds)
    # gen_tasks[task].append(ds_exps)

    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [400],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B', "google/gemma-2-9b"],
    'cache_path': ['./workdir/gen_output_loo_final'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_p1_upd1.sh')

{'mmlu': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'truthfullqa', 'pubmedqa', 'medquad']]}
['mmlu']


In [11]:
import numpy as np
import copy

n_exps = 1
tasks = ['sciq']#, 'truthfullqa', 'samsum', 'cnn', 'medquad']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    # for i, ds_tasks in enumerate(all_tasks):
    #     if task in ds_tasks:
    #         for ds in ds_tasks:
    #             if ds != task:
    #                 ds_exps.append(ds)
    # gen_tasks[task].append(ds_exps)

    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [400],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B', "google/gemma-2-9b"],
    'cache_path': ['./workdir/gen_output_loo_final'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_p1_upd2.sh')

{'sciq': [['samsum', 'xsum', 'cnn', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa', 'medquad']]}
['sciq']


In [46]:
import numpy as np
import copy

n_exps = 1
tasks = ['medquad']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    # for i, ds_tasks in enumerate(all_tasks):
    #     if task in ds_tasks:
    #         for ds in ds_tasks:
    #             if ds != task:
    #                 ds_exps.append(ds)
    # gen_tasks[task].append(ds_exps)

    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [100],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B', "google/gemma-2-9b"],
    'cache_path': ['./workdir/gen_output_loo_final_2'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps))
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_p1_1.sh')

{'medquad': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa']]}
['medquad']


In [48]:
import numpy as np
import copy

n_exps = 1
tasks = ['mmlu', 'sciq', 'samsum']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    # for i, ds_tasks in enumerate(all_tasks):
    #     if task in ds_tasks:
    #         for ds in ds_tasks:
    #             if ds != task:
    #                 ds_exps.append(ds)
    # gen_tasks[task].append(ds_exps)

    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [100],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B', "google/gemma-2-9b"],
    'cache_path': ['./workdir/gen_output_loo_final_2'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps))
}
    
cuda_devices = [1]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_p1_2.sh')

{'mmlu': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'truthfullqa', 'pubmedqa', 'medquad']], 'sciq': [['samsum', 'xsum', 'cnn', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa', 'medquad']], 'samsum': [['xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa', 'medquad']]}
['mmlu', 'sciq', 'samsum']


In [44]:
import numpy as np
import copy

n_exps = 1
tasks = ['gsm8k']#, 'cnn', 'samsum']#, 'medquad', 'xsum', 'pubmedqa', 'wmt14_fren', 'cnn', "wmt19_deen", 'samsum']
gen_tasks = {}
for k, task in enumerate(tasks):
    gen_tasks[task] = []
    ds_exps = []
    # for i, ds_tasks in enumerate(all_tasks):
    #     if task in ds_tasks:
    #         for ds in ds_tasks:
    #             if ds != task:
    #                 ds_exps.append(ds)
    # gen_tasks[task].append(ds_exps)

    ds_exps = []
    all_tasks_i = [x for x in np.concatenate(all_tasks) if x != task] 
    for i, ds in enumerate(all_tasks_i):
        ds_exps.append(ds)
    gen_tasks[task].append(ds_exps)
print(gen_tasks)
    
train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [400],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [2000],
    'model': ['meta-llama/Meta-Llama-3.1-8B', "google/gemma-2-9b"],
    'cache_path': ['./workdir/gen_output_loo'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps))
}
    
cuda_devices = [2]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_p2.sh')

{'gsm8k': [['samsum', 'xsum', 'cnn', 'sciq', 'coqa', 'triviaqa', 'mmlu', 'truthfullqa', 'pubmedqa', 'medquad']]}
['gsm8k']


# Mix of datasets

In [1]:
import yaml

def generate_train_str_config(config, task_name):
    config_list = []
    config_list.append('ignore_exceptions=False use_density_based_ue=True')
    
    config_list.append('batch_size={}'.format(config['batch_size']))
    config_list.append('train_test_split=False')
    config_list.append('subsample_train_dataset={}'.format(config['subsample_train_dataset']))
    config_list.append('subsample_background_train_dataset={}'.format(config['subsample_background_train_dataset']))
    config_list.append('subsample_eval_dataset={}'.format(config['subsample_eval_dataset']))
    config_list.append('model.path={}'.format(config['model']))
    if ("gemma" in config['model']) or ("mistral" in config['model'].lower()) or ("llama-3" in config['model'].lower()) or ("stablelm-2" in config['model'].lower()):
        config_list.append('+model.attn_implementation=eager')
    if ("cache_path" in config.keys()) and not config.get("upd_path", False):
        config_list.append('cache_path={}'.format(config['cache_path']))
    if ("samples_n" in config.keys()):
        config_list.append('+generation_params.samples_n={}'.format(config['samples_n']))
    if ("metric_thrs" in config.keys()):
        config_list.append('+metric_thrs={}'.format(config['metric_thrs']))
    if ("layers" in config.keys()):
        config_list.append('+layers={}'.format(config['layers'][config['model']]))
    if config.get("generalization", False):
        try:
            ood_cfg = train_exps[task_name][config["exp_idx"]]
        except:
            return ""
        for i, ds in enumerate(ood_cfg):
            if ("cache_path" in config.keys()) and config.get("upd_path", False):
                config_list.append('cache_path={}_on_{}'.format(config['cache_path'], ds))
            with open(f"../configs/polygraph_eval_{ds}.yaml") as stream:
                gen_config = yaml.safe_load(stream)
            config_list.append('+max_new_tokens_{}={}'.format(i+1, gen_config['max_new_tokens']))
            config_list.append('+train_dataset_{}=\"{}\"'.format(i+1, gen_config['dataset']))
            config_list.append('+train_text_column_{}={}'.format(i+1, gen_config['text_column']))
            config_list.append('+train_label_column_{}={}'.format(i+1, gen_config['label_column']))
            config_list.append('+train_prompt_{}=\"{}\"'.format(i+1, gen_config['prompt']))
            
            config_list[-1] = config_list[-1].replace("\n", "\\n")
            config_list[-1] = config_list[-1].replace("$", "\\$")
            config_list[-1] = config_list[-1].replace("'s", " is")
            config_list[-1] = config_list[-1].replace("(", "\\(")
            config_list[-1] = config_list[-1].replace(")", "\\)")
            config_list[-1] = config_list[-1].replace("}", "\\}")
            config_list[-1] = config_list[-1].replace("{", "\\{")
            config_list[-1] = config_list[-1].replace("]", "\\]")
            config_list[-1] = config_list[-1].replace("[", "\\[")
            config_list[-1] = config_list[-1].replace(",", "\\,")
            config_list[-1] = config_list[-1].replace("=", "\\=")
            # config_list[-1] = config_list[-1].replace("+", "\\+")
            config_list.append('+train_split_{}={}'.format(i+1, gen_config['train_split']))

            if "description" in gen_config.keys():
                config_list.append("+train_description_{}=\"{}\"".format(i+1, gen_config['description']))
                config_list[-1] = config_list[-1].replace("\n", "\\n")
                config_list[-1] = config_list[-1].replace("'s", " is")
                config_list[-1] = config_list[-1].replace("(", "\\(")
                config_list[-1] = config_list[-1].replace(")", "\\)")
                config_list[-1] = config_list[-1].replace("}", "\\}")
                config_list[-1] = config_list[-1].replace("{", "\\{")
                config_list[-1] = config_list[-1].replace(",", "\\,")
                config_list[-1] = config_list[-1].replace("$", "\\$")
                config_list[-1] = config_list[-1].replace("=", "\\=")
                config_list[-1] = config_list[-1].replace("]", "\\]")
                config_list[-1] = config_list[-1].replace("[", "\\[")
                # config_list[-1] = config_list[-1].replace("+", "\\+")
                
            if "few_shot_split" in gen_config.keys():
                config_list.append('+few_shot_split_{}={}'.format(i+1, gen_config['few_shot_split']))
            if "n_shot" in gen_config.keys():
                config_list.append('+train_n_shot_{}={}'.format(i+1, gen_config['n_shot']))

        try:
            ood_cfg = test_exps[task_name][config["exp_idx"]]
        except:
            return ""
        for i, ds in enumerate(ood_cfg):
            with open(f"../configs/polygraph_eval_{ds}.yaml") as stream:
                gen_config = yaml.safe_load(stream)
            config_list.append('+eval_max_new_tokens_{}={}'.format(i+1, gen_config['max_new_tokens']))
            config_list.append('+eval_dataset_{}=\"{}\"'.format(i+1, gen_config['dataset']))
            config_list.append('+eval_text_column_{}={}'.format(i+1, gen_config['text_column']))
            config_list.append('+eval_label_column_{}={}'.format(i+1, gen_config['label_column']))
            config_list.append('+eval_prompt_{}=\"{}\"'.format(i+1, gen_config['prompt']))
            
            config_list[-1] = config_list[-1].replace("\n", "\\n")
            config_list[-1] = config_list[-1].replace("$", "\\$")
            config_list[-1] = config_list[-1].replace("'s", " is")
            config_list[-1] = config_list[-1].replace("(", "\\(")
            config_list[-1] = config_list[-1].replace(")", "\\)")
            config_list[-1] = config_list[-1].replace("}", "\\}")
            config_list[-1] = config_list[-1].replace("{", "\\{")
            config_list[-1] = config_list[-1].replace("]", "\\]")
            config_list[-1] = config_list[-1].replace("[", "\\[")
            config_list[-1] = config_list[-1].replace(",", "\\,")
            config_list[-1] = config_list[-1].replace("=", "\\=")
            # config_list[-1] = config_list[-1].replace("+", "\\+")
            config_list.append('+eval_split_{}={}'.format(i+1, gen_config['train_split']))

            if "description" in gen_config.keys():
                config_list.append("+eval_description_{}=\"{}\"".format(i+1, gen_config['description']))
                config_list[-1] = config_list[-1].replace("\n", "\\n")
                config_list[-1] = config_list[-1].replace("'s", " is")
                config_list[-1] = config_list[-1].replace("(", "\\(")
                config_list[-1] = config_list[-1].replace(")", "\\)")
                config_list[-1] = config_list[-1].replace("}", "\\}")
                config_list[-1] = config_list[-1].replace("{", "\\{")
                config_list[-1] = config_list[-1].replace(",", "\\,")
                config_list[-1] = config_list[-1].replace("$", "\\$")
                config_list[-1] = config_list[-1].replace("=", "\\=")
                config_list[-1] = config_list[-1].replace("]", "\\]")
                config_list[-1] = config_list[-1].replace("[", "\\[")
                # config_list[-1] = config_list[-1].replace("+", "\\+")
                
            if "few_shot_split" in gen_config.keys():
                config_list.append('+eval_few_shot_split_{}={}'.format(i+1, gen_config['few_shot_split']))
            if "n_shot" in gen_config.keys():
                config_list.append('+eval_n_shot_{}={}'.format(i+1, gen_config['n_shot']))
    if ("baselines" in config.keys()):
        if config['baselines'] == "all":
            config_list.append('+run_baselines=False')
            config_list.append('+run_eigenscore=False')
            config_list.append('+md_device=cuda')
            config_list.append('+run_proposed_methods=True')
            config_list.append('+run_layerwise_methods=True')
            config_list.append('+clean_md_device=cuda')
        elif config['baselines']:
            config_list.append('+run_baselines=False')
            config_list.append('+run_eigenscore=False')
            config_list.append('+run_layerwise_methods=True')
            config_list.append('+md_device=cuda')
        else:
            config_list.append('+run_proposed_methods=True')
            config_list.append('+clean_md_device=cuda')
    if "is_ood" in config.keys():
        config_list.append('+is_ood={}'.format(config['is_ood']))
        
    return config_list

In [13]:
def generate_bash(configs, cuda_devices, tasks, generate_func, script_name="polygraph_eval", filename='', n_gpus=1):
    full_config = 'cd ../'
    j = 0
    print(tasks)
    n_devices = len(cuda_devices)
    for i, mc_configs in enumerate(configs):
        for conf in expand_config(mc_configs):
            for task_name in tasks:
                
                if (n_gpus == 1) or ((task_name not in ["gsm8k", "xsum", "medquad"]) and (("7b" in conf["model"]) or ("8b" not in conf["model"]))):
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python run_polygraph.py'
                else:
                    base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]},{cuda_devices[(j+1)%n_devices]} HYDRA_CONFIG=./configs/polygraph_eval_{task_name}.yaml python run_polygraph.py'
                    j+=1
                    
                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf, task_name))
                if not len(args.strip()):
                    continue
                new_task += f' {args}'
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'../scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [14]:
datasets_ts = ['samsum', 'xsum', 'cnn']
datasets_qa_s = ['sciq', 'coqa', 'triviaqa', 'mmlu']
datasets_qa_l = ['truthfullqa', 'pubmedqa', 'medquad']

np.random.seed(7777)

train_exps = {}
test_exps = {}
n_splits = 3
for split in range(n_splits):
    all_tasks_train = []
    all_tasks_test = []
    n_train = np.random.choice([1,2])
    ds1 = np.random.choice(datasets_ts, size=n_train, replace=False)
    all_tasks_train.extend(ds1)
    for ds in datasets_ts:
        if ds not in ds1:
            all_tasks_test.append(ds)
    
    ds1 = np.random.choice(datasets_qa_s, size=2, replace=False)
    all_tasks_train.extend(ds1)
    for ds in datasets_qa_s:
        if ds not in ds1:
            all_tasks_test.append(ds)

    n_train = np.random.choice([1,2])
    ds1 = np.random.choice(datasets_qa_l, size=n_train, replace=False)
    all_tasks_train.extend(ds1)
    for ds in datasets_qa_l:
        if ds not in ds1:
            all_tasks_test.append(ds)

    k = 0
    while all_tasks_test[k] in train_exps.keys():
        k += 1
    train_exps[all_tasks_test[k]] = [all_tasks_train]
    test_exps[all_tasks_test[k]] = [all_tasks_test]

In [15]:
train_exps

{'samsum': [['cnn', 'triviaqa', 'coqa', 'truthfullqa']],
 'sciq': [['cnn', 'xsum', 'mmlu', 'coqa', 'pubmedqa', 'truthfullqa']],
 'cnn': [['xsum', 'samsum', 'sciq', 'triviaqa', 'pubmedqa']]}

In [16]:
test_exps

{'samsum': [['samsum', 'xsum', 'sciq', 'mmlu', 'pubmedqa', 'medquad']],
 'sciq': [['samsum', 'sciq', 'triviaqa', 'medquad']],
 'cnn': [['cnn', 'coqa', 'mmlu', 'truthfullqa', 'medquad']]}

In [19]:
import numpy as np
import copy

n_exps = 1
tasks = [ds for ds in test_exps]

train_configs = {
    'batch_size': [1],
    'baselines': ["all"],
    'subsample_train_dataset': [500],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [500],
    'model': ['meta-llama/Meta-Llama-3.1-8B'],
    'cache_path': ['./workdir/gen_output_mix'],
    'samples_n': [5],
    'generalization': [True], 
    'metric_thrs': ["\"[0.3]\""],
    'layers': [layers],
    "exp_idx": list(range(n_exps)),
    "is_ood": [True],
}
    
cuda_devices = [0]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], filename='generalization/run_tmd_exps_mix.sh')

['samsum', 'sciq', 'cnn']


In [33]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = ["Replace me by any text you'd like."]
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input, output_hidden_states=True)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [34]:
len(output.hidden_states), output.hidden_states[-1].shape

(13, torch.Size([1, 12, 768]))

In [28]:
encoded_input['input_ids'].shape

torch.Size([1, 12])

In [32]:
import torch
torch.cat(output.hidden_states).shape

torch.Size([13, 12, 768])

In [35]:
layer = 1
output.hidden_states[layer].shape

torch.Size([1, 12, 768])